Идея решения простая -- преобразовать данные (перевести категориальные признаки в числовые и убрать " ?"), затем использовать xgboost с некоторыми подобранными параметрами. Однако в процессе тестирования на части train выборки как на тестовой выяснилось, что если результат регрессии xgboost'а переводить в 0 и 1 как "если ответ < 0.5, то это 0, а если > 0.5, то 1", то на тестовой выборке алгоритм чаще ошибается, относя ответы класса 1 к классу 0. Поэтому я пробовала переводить результат работы xgboost'а в бинарные ответы, взяв в качестве барьера числа от 0.35 до 0.5. Лучшее получилось на 0.4. В результате получился такой ноутбук:

In [5]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import svm, grid_search, datasets
import operator
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
%matplotlib inline

Считываем данные обучающей выборки и заменяем в каждом столбце "?" на самое часто встречающееся значение в этом столбце:

In [6]:
train_data = pd.read_csv('train.txt')
train_data = train_data.drop('id', axis=1)
train_label = pd.read_csv('target_train.txt', index_col='id')
replace_nans = {}
for column in train_data.columns:
    if train_data.dtypes[column] == np.object:
        replace_nans[column] = np.argmax(train_data[column].value_counts())
        train_data[column].replace(to_replace=' ?', value=replace_nans[column], inplace=True)
train_data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States


С помощью preprocessing.LabelEncoder() переводим катеориальные признаки в числовые:

In [7]:
def number_encode_features(train_data):
    result = train_data.copy()
    encoders = {}
    for column in result.columns:
        if result.dtypes[column] == np.object:
            encoders[column] = preprocessing.LabelEncoder()
            result[column] = encoders[column].fit_transform(result[column])
    return result, encoders

encoded_data, encoders = number_encode_features(train_data)
train_data = encoded_data
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,6,77516,9,13,4,0,1,4,1,2174,0,40,38
1,50,5,83311,9,13,2,3,0,4,1,0,0,13,38
2,38,3,215646,11,9,0,5,1,4,1,0,0,40,38
3,53,3,234721,1,7,2,5,0,2,1,0,0,40,38
4,28,3,338409,9,13,2,9,5,2,0,0,0,40,4


То же самое, что и с train_data, проделаем и с тестовой выборкой:

In [8]:
test_data_raw = test_data = pd.read_csv('test.txt')
test_data = test_data.drop('id', axis=1)

for column in test_data.columns:
    if test_data.dtypes[column] == np.object:
        test_data[column].replace(to_replace=' ?', value=replace_nans[column], inplace=True)

In [9]:
def encode_test(test_data):
    result = test_data.copy()
    for column in result.columns:
        if result.dtypes[column] == np.object:
            result[column] = encoders[column].transform(result[column])
    return result

test_data = encode_test(test_data)
X_test = np.array(test_data)
test_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,28,3,37821,8,11,4,11,4,4,0,0,0,55,38
1,45,3,228570,11,9,2,11,5,4,0,0,0,35,38
2,21,3,141453,11,9,4,7,3,4,0,0,0,40,38
3,34,3,88215,12,14,2,9,5,1,0,0,0,40,2
4,53,3,48641,2,8,4,7,1,3,0,0,0,35,38


Тут откуда-то в train_label нулевой индекс id, уберем его:

In [10]:
train_label = train_label[1:]

Функция, переводящая результат работы xgboost в бинарные ответы. Здесь ответы делятся значением 0.4:

In [11]:
def prediction(x, prob):
    return 0 if x <= prob else 1
prediction = np.vectorize(prediction)

Здесь 2 вида параметров для xgboost: мой лучший результат в контесте получен первым (по сути, почти default параметры), если же взять вторые (и, соответственно, поменять num_boost_round на 500 в обучении), то получится результат около 74.3 на private

In [12]:
params = {'base_score': 0.5, 'colsample_bylevel': 1, 'colsample_bytree': 1,
       'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3,
       'min_child_weight': 1, 'n_estimators': 1100, 'nthread': -1,
       'objective': 'binary:logistic', 'scale_pos_weight': 1, 'seed': 0, 'silent': True, 'subsample': 1}

In [14]:
params = {'silent': False, 'nthread': 10, 'max_depth': 8, 
                        'subsample': 0.5, 'learning_rate': 0.01, 'seed': 202, 'eta': 0.007}

In [15]:
dtrain = xgb.DMatrix(train_data, label=train_label)
dtest = xgb.DMatrix(test_data)

clf = xgb.train(params, dtrain, num_boost_round=200)
y_pred = clf.predict(dtest)

Переводим ответы в бинарные:

In [16]:
y = prediction(y_pred, 0.4)

In [17]:
pd.DataFrame(y, columns=['target'], index=test_data_raw.index).to_csv('y_n.csv')